<a href="https://colab.research.google.com/github/FrustratedBoy420/Project-Exhibition/blob/main/Diabetic_Patient_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/FrustratedBoy420/Project-Exhibition

Cloning into 'Project-Exhibition'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 7), reused 17 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1.63 MiB | 4.42 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata # Corrected: Added this import

# Step 1: GPU availability check
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Step 2: Hugging Face token ko Colab secrets se access karo
# Corrected: Fetching the token securely
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')

# Step 3: Model aur Tokenizer ko load karo
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected: Added token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True, # Recommended for memory efficiency
    load_in_8bit=True, # Recommended for memory efficiency
    token=huggingface_token # Corrected: Added token
)

print("Mistral-7B model and tokenizer loaded successfully.")

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Mistral-7B model and tokenizer loaded successfully.


In [4]:
import json

file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"Data se {len(chunks)} chunks mil gaye.")

Data se 638 chunks mil gaye.


In [5]:
from sentence_transformers import SentenceTransformer

# Embedding model ko load karo
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Chunks ka embeddings banao
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print("Embeddings successfully created.")
print(f"Pehle chunk ka embedding shape: {embeddings[0].shape}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Embeddings successfully created.
Pehle chunk ka embedding shape: (384,)


In [6]:
import numpy as np
import faiss

# Embeddings ko float32 format mein convert karo, jo FAISS ke liye zaruri hai
embeddings_np = np.array(embeddings).astype('float32')
dimension = embeddings_np.shape[1]

# FAISS index banao (yeh ek index hai jo fast search mein madad karta hai)
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Index ko save karo taki dobara na banana pade
faiss.write_index(index, "diabetes_faiss_index.bin")

print("FAISS index successfully created and saved.")

FAISS index successfully created and saved.


In [7]:
# Yeh code tumhare existing code ke niche aayega
from google.colab import userdata

def get_recommendation_from_doc(user_query, chunks):
    # 1. User ke sawal ka embedding banao
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # 2. Vector database mein sabse relevant chunks dhoondo (top 3)
    k = 3
    distances, indices = index.search(query_embedding_np, k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # 3. Mistral-7B ke liye prompt banao
    prompt = f"""
    [INST]
    Based on the following medical guidelines from the American Diabetes Association, answer the user's question. Make sure your answer is helpful, accurate, and concise.

    Medical Guidelines: {context}

    User Question: {user_query}
    [/INST]
    """

    # 4. Model se jawab generate karvao
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# Ab isko test karo!
user_query = "What are the physical activity recommendations for a diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, chunks)

print("\n--- Model's Recommendation ---")
print(f"Tumhara sawal: {user_query}")
print(f"Model ka jawab: {recommendation}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Model's Recommendation ---
Tumhara sawal: What are the physical activity recommendations for a diabetic patient?
Model ka jawab: Based on the American Diabetes Association's medical guidelines, physical activity is an essential part of diabetes management for all individuals with diabetes. The guidelines recommend a minimum of 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities at least two days a week. Regular physical activity can improve cardiovascular fitness, muscle strength, insulin sensitivity, and overall well-being. For those with type 2 diabetes, exercise has been shown to improve blood glucose levels, reduce cardiovascular risk factors, and contribute to weight loss.

     It's important to note that individualized medical nutrition therapy and diabetes self-management education are also crucial components of diabetes management. Frequent glucose monitoring and insulin dosing adjustments may be necessary to prevent hyp

##RAG TUNING


In [8]:
# ----------------------------------
# 1. Install Dependencies
# ----------------------------------
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import json


In [10]:
# ----------------------------------
# 2. Device Setup
# ----------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ----------------------------------
# 3. Load Model & Tokenizer (with quantization fix)
# ----------------------------------
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret

# Quantization config (8-bit to save VRAM, use 4-bit if still OOM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Changed to load_in_4bit=True
    bnb_4bit_quant_type="nf4", # Added 4-bit quantization type
    bnb_4bit_compute_dtype=torch.float16, # Added 4-bit compute dtype
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected use_auth_token to token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config,
    token=huggingface_token # Corrected use_auth_token to token
)

print("✅ Mistral-7B model and tokenizer loaded successfully.")

✅ Using device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Mistral-7B model and tokenizer loaded successfully.


In [11]:
# ----------------------------------
# 4. Load Knowledge Base (JSON file)
# ----------------------------------
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"✅ Loaded {len(chunks)} chunks from knowledge base")

# ----------------------------------
# 5. Create / Load FAISS Index
# ----------------------------------
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for chunks
embeddings = embedding_model.encode(chunks, show_progress_bar=True)
embeddings_np = np.array(embeddings).astype('float32')

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Save index
faiss.write_index(index, "diabetes_faiss_index.bin")
print("✅ FAISS index successfully created and saved.")

✅ Loaded 638 chunks from knowledge base


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

✅ FAISS index successfully created and saved.


In [12]:
# ----------------------------------
# 6. RAG Function
# ----------------------------------
def get_recommendation_from_doc(user_query, chunks, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# ----------------------------------
# 7. Test Query
# ----------------------------------
user_query = "What are the physical activity recommendations for a diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, chunks)

print("\n--- Model's Recommendation ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Model's Recommendation ---
❓ Question: What are the physical activity recommendations for a diabetic patient?
✅ Answer: 1. The American Diabetes Association (ADA) recommends a minimum of 150 minutes of moderate-intensity aerobic activity or 75 minutes of vigorous-intensity aerobic activity per week for all adults with diabetes, including those with type 1 and type 2 diabetes.
2. Both physical activity and exercise are essential for people with diabetes to improve physical fitness, insulin sensitivity, and overall well-being.
3. For those who do not meet the recommended activity guidelines, structured exercise programs are encouraged to improve physical activity levels.
4. Aerobic activity bouts should last at least 10 minutes to achieve cardiovascular fitness benefits and sustained improvements in cardiorespiratory fitness and weight loss.
5. Regular resistance and muscle-strengthening activities should be performed at least two days per week to improve overall fitness, bone densi